Accelerated Bayesian Causal Forest
https://johaupt.github.io/blog/xbcf.html

Accelerated Bayesian Causal Forest (XBCF) to estimate the conditional average treatment effect (or uplift) using a specialized version of Bayesian Additive Regression Trees (BART). It’s better described as Bayesian boosted trees for non-parametric causal inference.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xbcausalforest import XBCF

In [2]:
df = pd.read_csv("bpi2017_final.csv")

In [3]:
feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

In [11]:
# Split data to training and testing samples for model validation (next section)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11101)

In [12]:
treatment=df_train['treatmentOffer']
X = df_train[feature_names]
y=df_train['offerSuccess']

treatment_test=df_test['treatmentOffer']
X_test = df_test[feature_names]
y_test=df_test['offerSuccess']

In [13]:
NUM_TREES_PR  = 200
NUM_TREES_TRT = 100

cf = XBCF(
    #model="Normal",
    parallel=True, 
    num_sweeps=50, 
    burnin=15,
    max_depth=250,
    num_trees_pr=NUM_TREES_PR,
    num_trees_trt=NUM_TREES_TRT,
    num_cutpoints=100,
    Nmin=1,
    #mtry_pr=X1.shape[1], # default 0 seems to be 'all'
    #mtry_trt=X.shape[1], 
    tau_pr = 0.6 * np.var(y)/NUM_TREES_PR, #0.6 * np.var(y) / /NUM_TREES_PR,
    tau_trt = 0.1 * np.var(y)/NUM_TREES_TRT, #0.1 * np.var(y) / /NUM_TREES_TRT,
    alpha_pr= 0.95, # shrinkage (splitting probability)
    beta_pr= 2, # shrinkage (tree depth)
    alpha_trt= 0.95, # shrinkage for treatment part
    beta_trt= 2,
    p_categorical_pr = 0,
    p_categorical_trt = 0,
    standardize_target=True, # standardize y and unstandardize for prediction
         )

Since we specify the model as a sum of two BARTs, we can pass different sets of covariates to the outcome model and the treatment model, denoted by x and x_t. z is the treatment indicator coded 0/1.

In [14]:
cf.fit(
    x_t=X, # Covariates treatment effect
    x=X, # Covariates outcome (including propensity score)
    y=y,  # Outcome
    z=treatment, # Treatment group
)

XBCF(num_sweeps = 50, burnin = 15, max_depth = 250, Nmin = 1, num_cutpoints = 100, no_split_penality = 4.605170185988092, mtry_pr = 19, mtry_trt = 19, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 200, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 0.0007494867168461104, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 100, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = 0.00024982890561537014, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = False, random_seed = 0, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [15]:
tau_xbcf = cf.predict(X_test, return_mean=True)
tau_xbcf

array([-0.09794314,  0.14720209,  0.17394319, ...,  0.16667864,
       -0.09793973,  0.14136999])

In [17]:
%store -r df_results
lib = "xbcausalforest"
method = "Accelerated Bayesian Causal Forest"
ite = tau_xbcf
ate = tau_xbcf.mean()

if method in df_results['method'].values:
     # If the method is already in the DataFrame, update the ATE and ITE columns
    df_results.loc[df_results['method'] == method, 'ATE'] = ate
    df_results.loc[df_results['method'] == method, 'ITE'] = [ite]
    df_results.loc[df_results['method'] == method, 'Library'] = lib
else:
    # If the method is not in the DataFrame, add a new row
    df_results = df_results._append({'method': method, 'ATE': ate, 'ITE': ite, 'Library': lib}, ignore_index=True)

print(df_results)
%store df_results

                                method       ATE  \
0                    Linear Regression  0.449046   
1                            Double ML  0.471019   
2                                  IPW  0.311352   
3                            IPW Hajek  0.311352   
4                       IPW Stabalized  0.311352   
5            Propensity Score Matching -0.179289   
6                    Distance Matching  0.630322   
7                               IPW LR  0.149171   
8                            CausalEGM  0.124771   
9                          Causal Tree  0.034357   
10                         cforest_mse  0.087602   
11                        cforest_cmse  0.301697   
12                  cforest_cmse_p=0.5  0.032615   
13             cforest_cmse_p=0.5_md=3  0.042756   
14                       cforest_ttest  0.130985   
15  Accelerated Bayesian Causal Forest  0.049235   

                                                  ITE         Library  other  
0                                   